In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
import geopandas as gpd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from gelos.gelosdataset import GELOSDataSet
from gelos.gelosdatamodule import GELOSDataModule
import yaml
from gelos import config
from lightning.pytorch import Trainer


2025-11-26 00:49:48.996 | INFO     | gelos.config:<module>:11 - PROJ_ROOT path is: /app
/opt/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.8 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations
/opt/venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a sing

In [3]:
yaml_config_directory = config.PROJ_ROOT / 'gelos' / 'configs'
yaml_name = 'terramind_embedding_generation.yaml'
with open(yaml_config_directory / yaml_name, "r") as f:
    yaml_config = yaml.safe_load(f)
print(yaml.dump(yaml_config))

data:
  class_path: gelos.gelosdatamodule.GELOSDataModule
  init_args:
    bands:
      DEM:
      - DEM
      S1RTC:
      - VV
      - VH
      S2L2A:
      - COASTAL_AEROSOL
      - BLUE
      - GREEN
      - RED
      - RED_EDGE_1
      - RED_EDGE_2
      - RED_EDGE_3
      - NIR_BROAD
      - NIR_NARROW
      - WATER_VAPOR
      - SWIR_1
      - SWIR_2
    batch_size: 1
    num_workers: 0
    repeat_bands:
      DEM: 4
    target_size: 96
model:
  class_path: terratorch.tasks.EmbeddingGenerationTask
  init_args:
    embed_file_key: filename
    embedding_pooling: null
    has_cls: false
    layers:
    - -1
    model: terramind_v1_base
    model_args:
      merge_method: mean
      modalities:
      - S2L2A
      - S1RTC
      - DEM
      pretrained: true
    output_format: parquet
    temporal_cfg:
      temporal_pooling: keep
      temporal_wrapper: true
seed_everything: 0
trainer:
  accelerator: auto
  callbacks: []
  devices: auto
  max_epochs: 0
  num_nodes: 1
  strategy: aut

In [4]:
model_name = yaml_config['model']['init_args']['model']
output_dir = config.INTERIM_DATA_DIR / config.DATA_VERSION / model_name
output_dir.mkdir(exist_ok=True)

data_root = config.RAW_DATA_DIR / config.DATA_VERSION

# add variables to yaml config so it can be passed to classes
yaml_config['data']['init_args']['data_root'] = data_root
yaml_config['model']['init_args']['output_dir'] = output_dir

In [5]:
gelos_datamodule = GELOSDataModule(**yaml_config['data']['init_args'])

In [6]:
from terratorch.tasks import EmbeddingGenerationTask

In [7]:
task = EmbeddingGenerationTask(**yaml_config['model']['init_args'])

/opt/venv/lib/python3.11/site-packages/terratorch/tasks/embedding_generation.py:67: UserWarning: GeoTIFF selected; 2D token embeddings (ViT) will be reshaped to [C, sqrt(num_tokens), sqrt(num_tokens)] after dropping CLS if present.
  warnings.warn(


In [8]:
trainer = Trainer(accelerator='cpu')

INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [9]:
trainer.predict(model=task, datamodule=gelos_datamodule)

/opt/venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=95` in the `DataLoader` to improve performance.


Predicting DataLoader 0:   0%|          | 0/76822 [00:00<?, ?it/s]

ValueError: `file_ids` shape mismatch: expected (1, 4), got (1,)